In [13]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [14]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [15]:
## Split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [16]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [17]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataaset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [18]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [19]:
train_x.shape[1]

11

In [20]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [21]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [22]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [23]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 19s 431ms/step - loss: 31.2369 - root_mean_squared_error: 5.5890
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.9518 - root_mean_squared_error: 5.5634   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 30.2092 - root_mean_squared_error: 5.4963 - val_loss: 28.8324 - val_root_mean_squared_error: 5.3696

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 28.0269 - root_mean_squared_error: 5.2940
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.4606 - root_mean_squared_error: 5.3348 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.9442 - root_mean_squared_error: 5.2862 - val_loss: 26.6543 - val_root_mean_squared_error: 5.1628

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 184ms/step - loss: 27.3718 - root_mean_squared_error: 5.2318
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26.1070 - root_mean_squared_error: 5.

2025/10/31 21:42:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 18s 415ms/step - loss: 32.9219 - root_mean_squared_error: 5.7378
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.9204 - root_mean_squared_error: 5.5568   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 25.2818 - root_mean_squared_error: 5.0281 - val_loss: 16.2283 - val_root_mean_squared_error: 4.0284

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 16.6044 - root_mean_squared_error: 4.0748
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.0838 - root_mean_squared_error: 3.7481 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.2360 - root_mean_squared_error: 3.3520 - val_loss: 7.1834 - val_root_mean_squared_error: 2.6802

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 183ms/step - loss: 7.0355 - root_mean_squared_error: 2.6525
27/46 ━━━━━

2025/10/31 21:43:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 19s 424ms/step - loss: 30.5457 - root_mean_squared_error: 5.5268
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.0096 - root_mean_squared_error: 5.6576   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 31.5029 - root_mean_squared_error: 5.6127 - val_loss: 30.4354 - val_root_mean_squared_error: 5.5168

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 33.5105 - root_mean_squared_error: 5.7888
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.4212 - root_mean_squared_error: 5.5151 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.3553 - root_mean_squared_error: 5.4181 - val_loss: 28.3069 - val_root_mean_squared_error: 5.3204

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 29.5845 - root_mean_squared_error: 5.4392
42/46 ━

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 17s 381ms/step - loss: 32.5204 - root_mean_squared_error: 5.7027
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.9911 - root_mean_squared_error: 5.1824   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 18.5211 - root_mean_squared_error: 4.3036 - val_loss: 7.2943 - val_root_mean_squared_error: 2.7008

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 7.8126 - root_mean_squared_error: 2.7951
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.8737 - root_mean_squared_error: 2.4148 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.3098 - root_mean_squared_error: 2.0760 - val_loss: 2.9126 - val_root_mean_squared_error: 1.7066

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 2.7635 - root_mean_squared_error: 1.6624
37/46 ━━━━━━━

2025/10/31 21:43:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:51<00:00, 12.88s/trial, best loss: 1.5055618286132812]

2025/10/31 21:43:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.00036633222438624735), 'momentum': np.float64(0.7930211189252975)}
Best eval rmse: 1.5055618286132812


In [27]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/0af17c46633d43c28cadb1adbb53403c/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\ASUS\OneDrive\Desktop\MLFLOW\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[6.3854766],
       [7.1256332],
       [6.479869 ],
       ...,
       [6.014867 ],
       [6.8651466],
       [5.8540936]], shape=(1225, 1), dtype=float32)

In [28]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
2025/10/31 21:45:50 WARNING mlflow.tracking._model_registry.fluent: Run with id 0af17c46633d43c28cadb1adbb53403c has no artifacts at artifact path 'model', registering model based on models:/m-ba87284133fb4f52afbaac7b70450407 instead
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1761927350931, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761927350931, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-ba87284133fb4f52afbaac7b70450407', run_id='0af17c46633d43c28cadb1adbb53403c', step=0, timestamp=1761924506468, value=0.7725568413734436>], model_id='m-ba87284133fb4f52afbaac7b70450407', name='wine-quality', params={'lr': '0.07965201866539487', 'momentum': '0.23824741845218445'}, run_id='0af17c46633d43c28cadb1adbb53403c', run_link=None, source='models:/m-ba87284133fb4f52afbaac7b70450407', status='READY', status_message=None, tags={}, user_id=None, version=1>